# Module 

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
# from scripts.train_config import raw_data_dir

# Config

In [10]:
raw_data_dir = '../data/ml-1m/'

In [22]:
# #@save
# d2l.DATA_HUB['ml-100k'] = (
#     'http://files.grouplens.org/datasets/movielens/ml-100k.zip',
#     'cd4dcac4241c8a4ad7badc7ca635da8a69dddb83')

#@save
def read_data_ml100k():
#     data_dir = d2l.download_extract('ml-100k')
    names = ['user_id', 'item_id', 'rating', 'timestamp']
#     data = pd.read_csv(os.path.join(raw_data_dir, 'u.data'), '\t', names=names,
#                        engine='python')
#     names = ['user_id','gender','age','occupation','zip']
    data = pd.read_csv(os.path.join(raw_data_dir, 'ratings.dat'),sep='::',header=None,names=names)
    num_users = data.user_id.unique().shape[0]
    num_items = data.item_id.unique().shape[0]
    return data, num_users, num_items

#@save
def split_data_ml100k(data, num_users, num_items, split_mode='random',
                      test_ratio=0.1):
    """Split the dataset in random mode or seq-aware mode."""
    if split_mode == 'seq-aware':
        train_items, test_items, train_list = {}, {}, []
        for line in data.itertuples():
            u, i, rating, time = line[1], line[2], line[3], line[4]
            train_items.setdefault(u, []).append((u, i, rating, time))
            if u not in test_items or test_items[u][-1] < time:
                test_items[u] = (i, rating, time)
        for u in range(1, num_users + 1):
            train_list.extend(sorted(train_items[u], key=lambda k: k[3]))
        test_data = [(key, *value) for key, value in test_items.items()]
        train_data = [item for item in train_list if item not in test_data]
        train_data = pd.DataFrame(train_data)
        test_data = pd.DataFrame(test_data)
    else:
        mask = [
            True if x == 1 else False
            for x in np.random.uniform(0, 1, (len(data))) < 1 - test_ratio]
        neg_mask = [not x for x in mask]
        train_data, test_data = data[mask], data[neg_mask]
    return train_data, test_data
#@save
def load_data_ml100k(data, num_users, num_items, feedback='explicit'):
    users, items, scores = [], [], []
    inter = np.zeros((num_items, num_users)) if feedback == 'explicit' else {}
    for line in data.itertuples():
        user_index, item_index = int(line[1] - 1), int(line[2] - 1)
        score = int(line[3]) if feedback == 'explicit' else 1
        users.append(user_index)
        items.append(item_index)
        scores.append(score)
        if feedback == 'implicit':
            inter.setdefault(user_index, []).append(item_index)
        else:
            inter[item_index, user_index] = score
    return users, items, scores, inter

In [27]:
split_mode='seq-aware'
feedback='explicit'
test_ratio=0.1
# batch_size=256
    

data, num_users, num_items = read_data_ml100k()
train_data, test_data = split_data_ml100k(data, num_users, num_items,
                                          split_mode, test_ratio)

train_u, train_i, train_r, _ = load_data_ml100k(train_data, num_users,
                                                num_items, feedback)
test_u, test_i, test_r, _ = load_data_ml100k(test_data, num_users,
                                             num_items, feedback)

In [32]:
num_users

6040

In [33]:
num_items

3706

IndexError: index 3734 is out of bounds for axis 0 with size 3706

IndexError: index 3797 is out of bounds for axis 0 with size 3706

# Debug data 

In [3]:
import pandas as pd
import os
from scripts.train_config import cleaned_data_dir, cleaned_file_name

raw_data_dir = '../data/ml-1m/'

output_dir = '../data/debug'

raw_data_sep = b"::"

In [4]:
unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(os.path.join(raw_data_dir, 'users.dat'),sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(os.path.join(raw_data_dir, 'ratings.dat'),sep='::',header=None,names=rnames)

/Users/mac/opt/miniconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [5]:
user.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
user[user['user_id'].isin(range(100))]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
94,95,M,45,0,98201
95,96,F,25,16,78028
96,97,F,35,3,66210
97,98,F,35,7,33547


In [7]:
mnames = ['movie_id','title','genres']
movies = pd.read_csv(os.path.join(raw_data_dir, 'movies.dat'),sep='::',header=None,names=mnames, encoding='ISO-8859-1')

/Users/mac/opt/miniconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [8]:
! ls {raw_data_dir}

README      movies.dat  ratings.dat users.dat


In [9]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [24]:
def save_csv(df, file_path):
    df['target'] = df.apply(lambda row: '::'.join([str(ele) for ele in row.values]).replace('"', ""), axis=1)
    df[['target']].to_csv(file_path, index=False, header=None)

In [25]:
user_lst = range(100)

target_user = user[user['user_id'].isin(user_lst)]
target_ratings = ratings[ratings['user_id'].isin(user_lst)]
target_movies = movies[movies['movie_id'].isin(target_ratings['movie_id'].values)]

save_csv(target_user, os.path.join(output_dir, 'users.dat'))
save_csv(target_ratings, os.path.join(output_dir, 'ratings.dat'))
save_csv(target_movies, os.path.join(output_dir, 'movies.dat'))

# target_user.to_csv(os.path.join(output_dir, 'users.dat'), index=False, header=None, sep=raw_data_sep)

<ipython-input-24-711f5dd04adf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df.apply(lambda row: '::'.join([str(ele) for ele in row.values]).replace('"', ""), axis=1)


In [26]:
target_movies.head(12)

,movie_id,title,genres,target
0,1,Toy Story (1995),Animation|Children's|Comedy,1::Toy Story (1995)::Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy,2::Jumanji (1995)::Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance,3::Grumpier Old Men (1995)::Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama,4::Waiting to Exhale (1995)::Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy,5::Father of the Bride Part II (1995)::Comedy
5,6,Heat (1995),Action|Crime|Thriller,6::Heat (1995)::Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance,7::Sabrina (1995)::Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's,8::Tom and Huck (1995)::Adventure|Children's
8,9,Sudden Death (1995),Action,9::Sudden Death (1995)::Action
9,10,GoldenEye (1995),Action|Adventure|Thriller,10::GoldenEye (1995)::Action|Adventure|Thriller


<ipython-input-10-28e43aa7cdc1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df.apply(lambda row: '::'.join([str(ele) for ele in row.values]), axis=1)


In [23]:
save_csv(target_movies, os.path.join(output_dir, 'movies.dat'))


<ipython-input-22-711f5dd04adf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df.apply(lambda row: '::'.join([str(ele) for ele in row.values]).replace('"', ""), axis=1)


In [14]:
df = target_movies
df['target'] = df.apply(lambda row: '::'.join([str(ele) for ele in row.values]), axis=1)

<ipython-input-14-3f546aaad952>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df.apply(lambda row: '::'.join([str(ele) for ele in row.values]), axis=1)


In [16]:
df.head(14)

,movie_id,title,genres,target
0,1,Toy Story (1995),Animation|Children's|Comedy,1::Toy Story (1995)::Animation|Children's|Come...
1,2,Jumanji (1995),Adventure|Children's|Fantasy,2::Jumanji (1995)::Adventure|Children's|Fantas...
2,3,Grumpier Old Men (1995),Comedy|Romance,3::Grumpier Old Men (1995)::Comedy|Romance::3:...
3,4,Waiting to Exhale (1995),Comedy|Drama,4::Waiting to Exhale (1995)::Comedy|Drama::4::...
4,5,Father of the Bride Part II (1995),Comedy,5::Father of the Bride Part II (1995)::Comedy:...
5,6,Heat (1995),Action|Crime|Thriller,6::Heat (1995)::Action|Crime|Thriller::6::Heat...
6,7,Sabrina (1995),Comedy|Romance,7::Sabrina (1995)::Comedy|Romance::7::Sabrina ...
7,8,Tom and Huck (1995),Adventure|Children's,8::Tom and Huck (1995)::Adventure|Children's::...
8,9,Sudden Death (1995),Action,9::Sudden Death (1995)::Action::9::Sudden Deat...
9,10,GoldenEye (1995),Action|Adventure|Thriller,10::GoldenEye (1995)::Action|Adventure|Thrille...


In [72]:
'::'.join([str(ele) for ele in target_movies.loc[0].values])

"1::Toy Story (1995)::Animation|Children's|Comedy"